
# 4: Einführung in pandas: Transformationen



### Loading Packages

In [1]:
import pandas as pd
import numpy as np


---
### 1. Merge
- https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html#merge
- https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

#### Concat
> pandas provides various facilities for easily combining together Series and DataFrame objects with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

In [2]:
df = pd.DataFrame(np.random.randn(10, 4))
# break it into pieces
pieces = [df[:3], df[3:7], df[7:]]

In [3]:
pieces

[          0         1         2         3
 0 -0.575742 -0.927395  1.900035  0.671556
 1 -0.644529 -0.330046 -1.061264  0.788091
 2 -0.950063 -0.422629 -0.671923  0.439260,
           0         1         2         3
 3  0.597886  1.337740 -0.624538 -0.010075
 4  0.976944  0.284876  0.808131  1.025918
 5 -0.194821  0.395210  1.285161  0.256031
 6  0.596986 -0.813227  0.439992 -0.853272,
           0         1         2         3
 7  0.433395 -0.062636 -0.250862  0.877056
 8 -2.767200  0.666174  1.332334 -1.760658
 9 -1.053257 -0.572377  2.523239  0.586656]

In [4]:
pd.concat(pieces)

,0,1,2,3
0,-0.575742,-0.927395,1.900035,0.671556
1,-0.644529,-0.330046,-1.061264,0.788091
2,-0.950063,-0.422629,-0.671923,0.439260
3,0.597886,1.337740,-0.624538,-0.010075
4,0.976944,0.284876,0.808131,1.025918
5,-0.194821,0.395210,1.285161,0.256031
6,0.596986,-0.813227,0.439992,-0.853272
7,0.433395,-0.062636,-0.250862,0.877056
8,-2.767200,0.666174,1.332334,-1.760658
9,-1.053257,-0.572377,2.523239,0.586656


#### Join
- SQL style merges

In [5]:
left = pd.DataFrame({"key": ["foo", "foo", "bar"], "lval": [1, 2, 3]})
right = pd.DataFrame({"key": ["foo", "bar"], "rval": [4, 5]})
left

,key,lval
0,foo,1
1,foo,2
2,bar,3


In [6]:
right

,key,rval
0,foo,4
1,bar,5


In [7]:
#with merge (on column)
pd.merge(left, right, on="key")

,key,lval,rval
0,foo,1,4
1,foo,2,4
2,bar,3,5


In [8]:
#with join on column
left.join(right.set_index('key'), on='key')

,key,lval,rval
0,foo,1,4
1,foo,2,4
2,bar,3,5


In [9]:
#Join DataFrames using their indexes
left.join(right, lsuffix='_caller', rsuffix='_other')

,key_caller,lval,key_other,rval
0,foo,1,foo,4.0
1,foo,2,bar,5.0
2,bar,3,NaN,NaN


---
### 2. Grouping
https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html#grouping
> By “group by” we are referring to a process involving one or more of the following steps:
- Splitting the data into groups based on some criteria
- Applying a function to each group independently
- Combining the results into a data structure



#### groupby
> A groupby operation involves some combination of splitting the object, applying a function, and combining the results. This can be used to group large amounts of data and compute operations on these groups.<br>https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

In [10]:
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
df = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
                              'Parrot', 'Parrot'],
                   'Max Speed': [380., 370., 24., 26.]})
df

,Animal,Max Speed
0,Falcon,380.0
1,Falcon,370.0
2,Parrot,24.0
3,Parrot,26.0


In [11]:
# Grouping and then applying the mean() function to the resulting groups:
df.groupby(['Animal']).mean()

,Max Speed
Animal,
Falcon,375.0
Parrot,25.0


#### Aggregate

Aggregate using one or more operations over the specified axis



- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.aggregate.html
- https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#aggregation

In [12]:
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.aggregate.html
df = pd.DataFrame([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [np.nan, np.nan, np.nan]],
                    columns=['A', 'B', 'C'])

In [13]:
df

,A,B,C
0,1.0,2.0,3.0
1,4.0,5.0,6.0
2,7.0,8.0,9.0
3,NaN,NaN,NaN


In [14]:
#Aggregate these functions over the rows.
df.agg(['sum', 'min', 'mean'])

,A,B,C
sum,12.0,15.0,18.0
min,1.0,2.0,3.0
mean,4.0,5.0,6.0


In [15]:
#Aggregate over the columns.
df.agg("mean", axis="columns")

0    2.0
1    5.0
2    8.0
3    NaN
dtype: float64

---
### 3. Reshaping 
- https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html#reshaping
- https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

#### Stack

In [16]:
tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

In [17]:
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])

In [18]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])
df

A         B
first second                    
bar   one    -0.205633 -0.471385
      two    -0.259563  0.243044
baz   one    -1.559807 -0.434426
      two     0.114842  0.420016
foo   one     0.057673 -0.981344
      two    -0.269305  1.047086
qux   one    -0.167488  1.922684
      two    -0.822241 -1.415755

In [19]:
df2 = df[:4]
df2

A         B
first second                    
bar   one    -0.205633 -0.471385
      two    -0.259563  0.243044
baz   one    -1.559807 -0.434426
      two     0.114842  0.420016

In [20]:
#The stack() method “compresses” a level in the DataFrame’s columns:
stacked = df2.stack()
stacked

first  second   
bar    one     A   -0.205633
               B   -0.471385
       two     A   -0.259563
               B    0.243044
baz    one     A   -1.559807
               B   -0.434426
       two     A    0.114842
               B    0.420016
dtype: float64

#### Pivot Tables
https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#reshaping-pivot
> The function `pivot_table()` can be used to create spreadsheet-style pivot tables.

In [21]:
df = pd.DataFrame(
    {
        "A": ["one", "one", "two", "three"] * 3,
        "B": ["A", "B", "C"] * 4,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 2,
        "D": np.random.randn(12),
        "E": np.random.randn(12),
    }
)
df

,A,B,C,D,E
0,one,A,foo,0.262019,-0.760728
1,one,B,foo,-1.211660,0.736440
2,two,C,foo,-1.782464,-1.481559
3,three,A,bar,0.732068,0.429578
4,one,B,bar,-0.550970,-1.447965
5,one,C,bar,-0.073462,-0.039080
6,two,A,foo,-0.968117,0.109494
7,three,B,foo,0.281761,0.432133
8,one,C,foo,0.937222,0.442337
9,one,A,bar,-0.293167,0.211299


In [22]:
#We can produce pivot tables from this data very easily:
pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"])

C             bar       foo
A     B                    
one   A -0.293167  0.262019
      B -0.550970 -1.211660
      C -0.073462  0.937222
three A  0.732068       NaN
      B       NaN  0.281761
      C -0.012145       NaN
two   A       NaN -0.968117
      B  0.135072       NaN
      C       NaN -1.782464

####  Reshaping by melt

https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#reshaping-by-melt

- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.melt.html
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transform.html

<img src="https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_melt.png" width=44%> 
The top-level melt() function and the corresponding DataFrame.melt() are useful to massage a DataFrame into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are “unpivoted” to the row axis, leaving just two non-identifier columns, “variable” and “value”. The names of those columns can be customized by supplying the var_name and value_name parameters.<br>https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#reshaping-by-melt

In [23]:
df = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    }
)
df

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [24]:
df.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [25]:
df.melt(id_vars=["first", "last"], var_name="quantity")

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


#### Explode

transform list elements to rows

https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#exploding-a-list-like-column

In [26]:
#Sometimes the values in a column are list-like:
df = pd.DataFrame({'A': [[0, 1, 2], 'foo', [], [3, 4]],
                   'B': 1,
                   'C': [['a', 'b', 'c'], np.nan, [], ['d', 'e']]})
df

,A,B,C
0,"[0, 1, 2]",1,"[a, b, c]"
1,foo,1,NaN
2,[],1,[]
3,"[3, 4]",1,"[d, e]"


In [27]:
#We can ‘explode’ the values column, transforming each list-like to a separate row, by using explode(). 
#This will replicate the index values from the original row:
df.explode('A')

,A,B,C
0,0,1,"[a, b, c]"
0,1,1,"[a, b, c]"
0,2,1,"[a, b, c]"
1,foo,1,NaN
2,NaN,1,[]
3,3,1,"[d, e]"
3,4,1,"[d, e]"
